In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
import gensim 
from gensim.models import Word2Vec 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [4]:
from nltk.corpus import stopwords
breakers = {'with', 'without', 'comma', 'and', 'or', 'by', 'in', 'due', 'to', 'of', 'causing'}
stop_words= (set(stopwords.words("english")) | {'nan', 'unspecified', 'stated', 'other', 'state'})

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [7]:
from sklearn.preprocessing import OneHotEncoder
import itertools

In [8]:
df_train = np.asarray(pd.read_csv("data/diabetes/diab_train.csv", skiprows=1, header=None))
df_val = np.asarray(pd.read_csv("data/diabetes/diab_validation.csv", skiprows=1, header=None))
df_test = np.asarray(pd.read_csv("data/diabetes/diab_test.csv", skiprows=1, header=None))

In [9]:
X = np.hstack([df_train[:,:-4], df_train[:,-3:]])
X_val = np.hstack([df_val[:,:-4], df_val[:,-3:]])
X_test = np.hstack([df_test[:,:-4], df_test[:,-3:]])
Y = df_train[:,-4]
Y_val = df_val[:,-4]
Y_test = df_test[:,-4]

In [10]:
X_all = np.vstack([X, X_val, X_test])

In [55]:
def sanitize(doc, breaks=True):
    return [lem.lemmatize(w) for w in tokenizer.tokenize(str(doc).replace(",", " comma").lower())\
                     if w not in stop_words or (breaks and w in breakers)]

In [110]:
def onehot(keys, tokens_docs, truncate=100):
    # convert list of of token-lists to one flat list of tokens
    # and then create a dictionary that maps word to id of word,
    all_tokens = itertools.chain.from_iterable(tokens_docs)
    word_to_id = {token: idx for idx, token in enumerate(set(all_tokens))}

    # convert token lists to token-id lists
    token_ids = [[word_to_id[token] for token in tokens_doc] for tokens_doc in tokens_docs]

    maxl = 0
    maxi = 0
    for t in token_ids:
        maxl = max(maxl, len(t))
        for w in t:
            maxi = max(maxi, w)

    mapping = {}
    for i,t in enumerate(token_ids):
        enc = np.zeros((min(maxl, truncate), maxi+1))
        for j in range(min(len(t), truncate)):
            enc[j][t[j]] += 1
        mapping[keys[i]] = enc
    return mapping

In [572]:
def encode(index, mapping, force_string=False):
    Xp = X_all[:,index]
    
    if force_string:
        Xp = Xp.astype(str)

    return np.asarray([mapping[x] for x in Xp])

In [573]:
def identity_map(data):
    return {x:x for x in set(data)}

In [574]:
encodings = []

Race

In [575]:
print(set(X_all[:,1]))

{'Hispanic', 'Other', 'Asian', 'Caucasian', '?', 'AfricanAmerican'}


In [576]:
race = {'Other': 1, 'Asian':2, 'Caucasian':3, 'Hispanic':4, 'AfricanAmerican':5, '?':0}

In [577]:
encodings.append(race)

Gender

In [578]:
print(set(X_all[:,2]))

{'Female', 'Male'}


In [579]:
gender = {'Male': 0, 'Female': 1}

In [580]:
encodings.append(gender)

Age

In [581]:
print(set(X_all[:,3]))

{'[50-60)', '[80-90)', '[10-20)', '[20-30)', '[30-40)', '[90-100)', '[0-10)', '[70-80)', '[40-50)', '[60-70)'}


In [582]:
age = {'[0-10)': 0, '[80-90)': 8, '[60-70)': 6, '[50-60)': 5,\
          '[30-40)': 3, '[20-30)':2, '[90-100)':9, '[40-50)':4, '[70-80)': 7, '[10-20)': 10}

In [583]:
encodings.append(age)

Weight

In [584]:
print(set(X_all[:,4]))

{'[100-125)', '[0-25)', '[75-100)', '[125-150)', '?', '[25-50)', '[150-175)', '[50-75)'}


In [585]:
weight = {'[25-50)': 25, '[150-175)': 150, '[75-100)': 75,\
          '[0-25)': 0, '[125-150)': 125, '[50-75)': 50, '[100-125)':100, '?': 73}

In [586]:
encodings.append(weight)

Admission Type ID

In [587]:
print(set(X_all[:,5]))

{nan, 'Urgent', 'Elective', 'Newborn', 'Not Mapped', 'Not Available', 'Emergency'}


In [588]:
admission_type = {'nan': 0, 'Newborn': 2, 'Emergency':4,\
                  'Not Mapped': 0, 'Not Available': 0, 'Urgent': 3, 'Elective': 1}

In [643]:
X_all[:,5] = X_all[:,5].astype(str)

In [590]:
encodings.append(admission_type)

Discharge Disposition ID

In [591]:
print(len(set(X_all[:,6])), 'Items')
print(sorted(list(set(X_all[:,6].astype(str)))))

22 Items
['Admitted as an inpatient to this hospital', 'Discharged to home', 'Discharged/transferred to ICF', 'Discharged/transferred to SNF', 'Discharged/transferred to a federal health care facility.', 'Discharged/transferred to a long term care hospital.', 'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare', 'Discharged/transferred to another  type of inpatient care institution', 'Discharged/transferred to another rehab fac including rehab units of a hospital.', 'Discharged/transferred to another short term hospital', 'Discharged/transferred to home under care of Home IV provider', 'Discharged/transferred to home with home health service', 'Discharged/transferred within this institution to Medicare approved swing bed', 'Discharged/transferred/referred another institution for outpatient services', 'Discharged/transferred/referred to a psychiatric hospital of a psychiatric distinct part unit of a hospital', 'Discharged/transferred/r

In [592]:
discharge_disposition = sorted([sanitize(s, breaks=False) for s in set(X_all[:,6].astype(str))])
discharge_disposition_onehot = onehot([s for s in set(X_all[:,6].astype(str))], discharge_disposition, truncate=4)

In [593]:
X_6, X_val_6, X_test_6 = encode(6, discharge_disposition_onehot, force_string=True)

ValueError: too many values to unpack (expected 3)

Admission Source ID

In [594]:
print(sorted(list(set(X_all[:,7].astype(str)))))

['Clinic Referral', 'Court/Law Enforcement', 'Emergency Room', 'HMO Referral', 'Not Available', 'Not Mapped', 'Physician Referral', 'Transfer from a Skilled Nursing Facility (SNF)', 'Transfer from a hospital', 'Transfer from another health care facility', 'nan']


In [656]:
admission_src_id = {'Clinic Referral':np.asarray([1,0,0,0,0,0,0,0,0]), \
                    'Court/Law Enforcement':np.asarray([0,1,0,0,0,0,0,0,0]), \
                    'Emergency Room':np.asarray([0,0,1,0,0,0,0,0,0]), \
                    'HMO Referral':np.asarray([0,0,0,1,0,0,0,0,0]), \
                    'Not Available':np.asarray([0,0,0,0,1,0,0,0,0]), \
                    'Not Mapped':np.asarray([0,0,0,0,1,0,0,0,0]), \
                    'Physician Referral':np.asarray([0,0,0,0,0,1,0,0,0]), \
                    'Transfer from a Skilled Nursing Facility (SNF)':np.asarray([0,0,0,0,0,0,1,0,0]), \
                    'Transfer from a hospital':np.asarray([0,0,0,0,0,0,0,1,0]), \
                    'Transfer from another health care facility':np.asarray([0,0,0,0,0,0,0,0,1])}

In [657]:
X_all[:,7] = X_all[:,7].astype(str)

In [658]:
encodings.append(admission_src_id)

Time in Hospital

In [598]:
print(set(X_all[:,8]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [599]:
encodings.append(identity_map(X_all[:,8]))

Payer Code

In [600]:
print(set(X_all[:,9]))

{'DM', 'CP', 'CM', 'SI', 'CH', 'OG', 'PO', 'OT', 'MC', '?', 'UN', 'BC', 'WC', 'HM', 'SP', 'MD'}


In [601]:
payer_code = {k:i for i,k in enumerate(set(X_all[:,9]))}

In [602]:
encodings.append(payer_code)

Medical Speciality

In [603]:
print(sorted([s for s in set(X_all[:,10])]))

['?', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-Pulmonology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Psychiatry', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-Plasticw

In [604]:
medical_speciality = sorted([sanitize(s, breaks=False) for s in set(X_all[:,10])])
medical_speciality_onehot = onehot([s for s in set(X_all[:,10])], medical_speciality)

In [605]:
X_10, X_val_10, X_test_10 = encode(10, medical_speciality_onehot)

ValueError: too many values to unpack (expected 3)

Number of Lab Procedures

In [606]:
print(set(X_all[:,11]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 103, 104, 106, 107, 108, 109, 113, 114, 120}


In [607]:
encodings.append(identity_map(X_all[:,11]))

Number of Procedures

In [608]:
print(set(X_all[:,12]))

{0, 1, 2, 3, 4, 5, 6}


In [609]:
encodings.append(identity_map(X_all[:,12]))

Number of Medications

In [610]:
print(set(X_all[:,13]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 67, 74, 75, 81}


In [611]:
encodings.append(identity_map(X_all[:,13]))

Number Outpatient

In [612]:
print(set(X_all[:,14]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 27, 36}


In [613]:
encodings.append(identity_map(X_all[:,14]))

Number Emergency

In [614]:
print(set(X_all[:,15]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 42, 13}


In [615]:
encodings.append(identity_map(X_all[:,15]))

Number Inpatient

In [616]:
print(set(X_all[:,16]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [617]:
encodings.append(identity_map(X_all[:,16]))

Diagnoses

In [618]:
print(set(X_all[:,17])|set(X_all[:,18])|set(X_all[:,19]))

{'31', '860', '618', '920', '906', '977', '42', '218', '258', '890', '70', '995', '442', '307', '716', 'V43', 'E949', 'V53', '720', '344', '416', '822', '351', '656', '246', '250.7', '183', '519', '250.82', '459', 'E929', '389', '161', '923', '317', '201', 'V63', '527', '582', '250.5', '440', '935', '?', '714', '308', 'E980', '242', '669', '537', '813', '455', '263', 'E937', '82', '751', '713', 'V25', '458', 'V18', '197', '969', '372', '200', '999', '376', '210', '427', '198', '591', '388', '594', '759', '536', '274', '966', '682', '918', '292', '481', 'E927', 'V12', '530', '736', 'E816', '965', '301', 'V14', '533', '88', '250.43', '991', '385', 'E932', '199', 'E878', 'V56', '691', '250.4', '160', '567', '155', '245', '239', 'V27', '745', '470', 'E939', '214', 'V65', '574', '250.12', '620', '711', '350', '365', 'V58', '185', '741', '614', '374', 'E934', '171', '706', '595', '456', '611', '581', '868', '275', '189', '695', '436', '723', '187', '309', '203', '627', '173', '781', '394', '

Number of Diagnoses

In [619]:
print(set(X_all[:,20]))

{1, 2, 3, 4, 5, 6, 7, 8, 9}


In [620]:
encodings.append(identity_map(X_all[:,20]))

Max Glucose Serum

In [621]:
print(set(X_all[:,21]))

{'Norm', '>200', '>300', 'None'}


In [622]:
max_glu_serum = {'Norm':0, '>300':3, 'None':1, '>200':2}

In [623]:
encodings.append(max_glu_serum)

A1Cresult

In [624]:
print(set(X_all[:,22]))

{'Norm', '>8', '>7', 'None'}


In [625]:
a1c_res = {'>7': 2, 'Norm': 0, '>8': 3, 'None': 1}

In [626]:
encodings.append(a1c_res)

Metformin - Metformin Pioglitazone

In [627]:
print(set(X_all[:,23]))

{'Down', 'Steady', 'No', 'Up'}


In [628]:
drug_change = {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3}

In [629]:
encodings += [drug_change] * 23

Change

In [630]:
print(set(X_all[:,46]))

{'No', 'Ch'}


In [631]:
change = {'No': 0, 'Ch': 1}

In [632]:
encodings.append(change)

Diabetes Medication

In [633]:
print(set(X_all[:,47]))

{'No', 'Yes'}


In [634]:
diabetes_med = {'No': 0, 'Yes': 1}

In [635]:
encodings.append(diabetes_med)

Diagnoses Description

In [636]:
print(set(X_all[:,48]))

{nan, 'Closed fracture of upper end of forearm, unspecified', 'Postmastectomy lymphedema syndrome', 'Hereditary peripheral neuropathy', 'Chronic tonsillitis', 'Chronic pyelonephritis without lesion of renal medullary necrosis', 'Anal and rectal polyp', 'Closed fracture of vault of skull without mention of intracranial injury, unspecified state of consciousness', 'Autoimmune hemolytic anemias', 'Unspecified cardiac device in situ', 'Internal hemorrhoids without mention of complication', 'Cellulitis and abscess of finger, unspecified', 'Allergic urticaria', 'Gouty arthropathy, unspecified', 'Regional enteritis of small intestine', 'Spontaneous tension pneumothorax', 'Other degenerative diseases of the basal ganglia', 'Diabetes mellitus of mother, complicating pregnancy, childbirth, or the puerperium, unspecified as to episode of care or not applicable', 'Dyschromia, unspecified', 'Maternal distress complicating labor and delivery, unspecified as to episode of care or not applicable', 'Ce

In [637]:
diagnoses1 = sorted([sanitize(s) for s in set(X_all[:,48])])
diagnoses1_onehot = onehot([str(s) for s in set(X_all[:,48])], diagnoses1, truncate=10)

In [638]:
diagnoses2 = sorted([sanitize(s) for s in set(X_all[:,49])])
diagnoses2_onehot = onehot([str(s) for s in set(X_all[:,49])], diagnoses2, truncate=10)

In [639]:
diagnoses3 = sorted([sanitize(s) for s in set(X_all[:,50])])
diagnoses3_onehot = onehot([str(s) for s in set(X_all[:,50])], diagnoses3, truncate=10)

In [640]:
X_all_enc = np.zeros((len(X_all), 42))

In [641]:
len(encodings)

42

In [645]:
for i in range(5):
    print(i, encodings[i])
    X_all_enc[:,i] = encode(1+i, encodings[i])

0 {'Other': 1, 'Asian': 2, 'Caucasian': 3, 'Hispanic': 4, 'AfricanAmerican': 5, '?': 0}
1 {'Male': 0, 'Female': 1}
2 {'[0-10)': 0, '[80-90)': 8, '[60-70)': 6, '[50-60)': 5, '[30-40)': 3, '[20-30)': 2, '[90-100)': 9, '[40-50)': 4, '[70-80)': 7, '[10-20)': 10}
3 {'[25-50)': 25, '[150-175)': 150, '[75-100)': 75, '[0-25)': 0, '[125-150)': 125, '[50-75)': 50, '[100-125)': 100, '?': 73}
4 {'nan': 0, 'Newborn': 2, 'Emergency': 4, 'Not Mapped': 0, 'Not Available': 0, 'Urgent': 3, 'Elective': 1}


In [646]:
for i in range(3):
    print(5+i, encodings[5+i], X[:,7+i][:5])
    X_all_enc[:,5+i] = encode(7+i, encodings[5+i])

5 {'Clinic Referral': 0, 'Court/Law Enforcement': 1, 'Emergency Room': 2, 'HMO Referral': 3, 'Not Available': 4, 'Not Mapped': 5, 'Physician Referral': 6, 'Transfer from a Skilled Nursing Facility (SNF)': 7, 'Transfer from a hospital': 8, 'Transfer from another health care facility': 9, 'nan': 10} ['Emergency Room' 'Physician Referral' 'Emergency Room' 'Emergency Room'
 'Physician Referral']
6 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14} [4 1 2 4 13]
7 {'DM': 0, 'CP': 1, 'CM': 2, 'SI': 3, 'CH': 4, 'OG': 5, 'PO': 6, 'OT': 7, 'MC': 8, '?': 9, 'UN': 10, 'BC': 11, 'WC': 12, 'HM': 13, 'SP': 14, 'MD': 15} ['MC' '?' 'MC' 'DM' '?']


In [647]:
for i in range(6):
    print(8+i, encodings[8+i], X[:,11+i][:5])
    X_all_enc[:,8+i] = encode(11+i, encodings[8+i])

8 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 101: 101, 103: 103, 104: 104, 106: 106, 107: 107, 108: 108, 109: 109, 113: 113, 114: 114, 120: 120} [40 24 59 60 54]
9 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6} [4 1 0 1 6]
10 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9,

In [648]:
for i in range(28):
    print(14+i, encodings[14+i], X[:,20+i][:5])
    X_all_enc[:,14+i] = encode(20+i, encodings[14+i])

14 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9} [9 3 8 5 6]
15 {'Norm': 0, '>300': 3, 'None': 1, '>200': 2} ['None' 'None' 'None' 'None' 'None']
16 {'>7': 2, 'Norm': 0, '>8': 3, 'None': 1} ['None' 'None' 'None' 'None' 'None']
17 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'Steady' 'No' 'No' 'No']
18 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
19 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
20 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
21 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
22 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
23 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
24 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'Steady' 'No' 'No' 'Down']
25 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'No' 'No' 'No' 'No']
26 {'Down': 1, 'No': 0, 'Steady': 2, 'Up': 3} ['No' 'Steady' 'No' 'No' 'No']
27 {'Down': 1, 'No':

In [569]:
X_enc.shape

(6000, 42)

In [570]:
len(encodings)

42

In [651]:
from sklearn.preprocessing import Normalizer
X_all_enc = Normalizer().fit_transform(X_all_enc)

In [652]:
X_enc = X_all_enc[:len(X)]
X_val_enc = X_all_enc[len(X):len(X)+len(X_val)]
X_test_enc = X_all_enc[len(X)+len(X_val):]

In [655]:
np.savetxt("data/diabetes/X_enc.csv", X_enc, delimiter=",")
np.savetxt("data/diabetes/X_val_enc.csv", X_val_enc, delimiter=",")
np.savetxt("data/diabetes/X_test_enc.csv", X_test_enc, delimiter=",")